Install the necessary libraries 

In [1]:
!pip install implicit

In [2]:
import pandas as pd 
import numpy as np 
import implicit
import scipy.sparse as sparse

In [3]:
#Read in dataset
df_raw = pd.read_excel("./data/Online Retail.xlsx")

In [4]:
df_raw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 541909 entries, 0 to 541908
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   InvoiceNo    541909 non-null  object        
 1   StockCode    541909 non-null  object        
 2   Description  540455 non-null  object        
 3   Quantity     541909 non-null  int64         
 4   InvoiceDate  541909 non-null  datetime64[ns]
 5   UnitPrice    541909 non-null  float64       
 6   CustomerID   406829 non-null  float64       
 7   Country      541909 non-null  object        
dtypes: datetime64[ns](1), float64(2), int64(1), object(4)
memory usage: 33.1+ MB


In [5]:
#Dropping records with no CustomerID
df_raw.dropna(subset=['CustomerID'],
                      inplace=True)

In [6]:
df_raw['CustomerID'] = df_raw['CustomerID'].astype('int64')
df_raw['StockCode'] = df_raw['StockCode'].astype('str')

In [7]:
df_raw['Sales'] = df_raw['Quantity'] * df_raw['UnitPrice']

In [8]:
df_raw.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 406829 entries, 0 to 541908
Data columns (total 9 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   InvoiceNo    406829 non-null  object        
 1   StockCode    406829 non-null  object        
 2   Description  406829 non-null  object        
 3   Quantity     406829 non-null  int64         
 4   InvoiceDate  406829 non-null  datetime64[ns]
 5   UnitPrice    406829 non-null  float64       
 6   CustomerID   406829 non-null  int64         
 7   Country      406829 non-null  object        
 8   Sales        406829 non-null  float64       
dtypes: datetime64[ns](1), float64(2), int64(2), object(4)
memory usage: 31.0+ MB


## Filter out records on for Customers who bought more than n items

In [9]:
df_items_per_cust = df_raw.groupby(['CustomerID'])\
.agg({'StockCode': 'nunique'}).reset_index()

In [10]:
df_items_per_cust.columns = ['CustomerID', 'Count_item_cust']

In [11]:
#Setting of THreshold
item_in_cust_threshold = 6

In [12]:
#Filtering Results
mask = df_items_per_cust['Count_item_cust'] >= item_in_cust_threshold
valid_cust= set(df_items_per_cust.loc[mask, 'CustomerID'].tolist())

In [13]:
df_filter_cust = df_raw[df_raw['CustomerID'].isin(valid_cust)].copy()

In [14]:
invoiceno_filter_cust = set(df_filter_cust['InvoiceNo'].tolist())

## Filter out items only for those that are bought by multiple customers

In [15]:
df_custs_per_item = df_raw.groupby(['StockCode'])\
.agg({'CustomerID': 'nunique'}).reset_index()

In [16]:
df_custs_per_item.columns = ['StockCode', 'Count_cust_item']

In [17]:
df_custs_per_item['Count_cust_item'].value_counts()

1      189
2      145
3      116
6       83
5       81
      ... 
407      1
274      1
319      1
256      1
379      1
Name: Count_cust_item, Length: 379, dtype: int64

In [18]:
# Set threshold 
cust_in_item_threshold = 6

In [19]:
mask = df_custs_per_item['Count_cust_item'] >= cust_in_item_threshold
valid_stockcode = set(df_custs_per_item.loc[mask, 'StockCode'].tolist())

In [20]:
df_filter_item = df_raw[df_raw['StockCode'].isin(valid_stockcode)].copy()

In [21]:
invoiceno_filter_item = set(df_filter_item['InvoiceNo'].tolist())

In [22]:
invoiceno_intersect = set.intersection(invoiceno_filter_item,invoiceno_filter_cust)

In [23]:
print(f"No. of invoice after filtering customer: {len(invoiceno_filter_cust)}")
print(f"No. of invoice after filtering item: {len(invoiceno_filter_item)}")
print(f"No. of invoice from intersect: {len(invoiceno_intersect)}")

No. of invoice after filtering customer: 21604
No. of invoice after filtering item: 22136
No. of invoice from intersect: 21557


In [24]:
df_filter_cust_item = df_raw[df_raw['InvoiceNo'].isin(invoiceno_intersect)].copy()

In [25]:
df_filter_cust_item.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 405521 entries, 0 to 541908
Data columns (total 9 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   InvoiceNo    405521 non-null  object        
 1   StockCode    405521 non-null  object        
 2   Description  405521 non-null  object        
 3   Quantity     405521 non-null  int64         
 4   InvoiceDate  405521 non-null  datetime64[ns]
 5   UnitPrice    405521 non-null  float64       
 6   CustomerID   405521 non-null  int64         
 7   Country      405521 non-null  object        
 8   Sales        405521 non-null  float64       
dtypes: datetime64[ns](1), float64(2), int64(2), object(4)
memory usage: 30.9+ MB


## Apply Collaborative Filtering with Implicit LIbrary

In [26]:
unique_customers = df_filter_cust_item['CustomerID'].unique()
cust_ids = dict(zip(unique_customers, np.arange(unique_customers.shape[0], dtype=np.int32)))

unique_items = df_filter_cust_item['StockCode'].unique()
item_ids = dict(zip(unique_items, np.arange(unique_items.shape[0], dtype=np.int32)))

df_filter_cust_item['cust_id'] = df_filter_cust_item['CustomerID'].apply(lambda i : cust_ids[i])
df_filter_cust_item['item_id'] = df_filter_cust_item['StockCode'].apply(lambda i : item_ids[i])

In [27]:
print(f"{len(cust_ids)}, {len(item_ids)}")

4017, 3671


In [28]:
df_filter_cust_item.sort_values(by=['StockCode'], inplace=True)
df_filter_cust_item.head(10)

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,Sales,cust_id,item_id
143930,548714,10002,INFLATABLE POLITICAL GLOBE,2,2011-04-03 15:07:00,0.85,17337,United Kingdom,1.70,1329,31
160128,550452,10002,INFLATABLE POLITICAL GLOBE,1,2011-04-18 12:56:00,0.85,14525,United Kingdom,0.85,679,31
5466,536863,10002,INFLATABLE POLITICAL GLOBE,1,2010-12-03 11:19:00,0.85,17967,United Kingdom,0.85,202,31
75792,542610,10002,INFLATABLE POLITICAL GLOBE,14,2011-01-30 14:05:00,0.85,13148,United Kingdom,11.90,602,31
20617,538069,10002,INFLATABLE POLITICAL GLOBE,8,2010-12-09 14:08:00,0.85,16795,United Kingdom,6.80,544,31
21507,538086,10002,INFLATABLE POLITICAL GLOBE,10,2010-12-09 14:44:00,0.85,12872,United Kingdom,8.50,553,31
21551,538093,10002,INFLATABLE POLITICAL GLOBE,12,2010-12-09 14:49:00,0.85,12682,France,10.20,217,31
77513,542735,10002,INFLATABLE POLITICAL GLOBE,12,2011-01-31 15:36:00,0.85,12681,France,10.20,380,31
22380,538167,10002,INFLATABLE POLITICAL GLOBE,12,2010-12-09 18:58:00,0.85,14713,United Kingdom,10.20,575,31
23275,538196,10002,INFLATABLE POLITICAL GLOBE,36,2010-12-10 10:56:00,0.85,12731,France,30.60,588,31


In [29]:
df_cust_item_qty = df_filter_cust_item.groupby(['cust_id','item_id'])\
.agg({'Quantity':'sum'}).reset_index()

In [30]:
df_cust_item_qty.head()

,cust_id,item_id,Quantity
0,0,0,122
1,0,1,122
2,0,2,108
3,0,3,110
4,0,4,104


In [31]:
#Create Sparse Matrix

sparse_customer_item = \
sparse.csr_matrix((df_cust_item_qty['Quantity'].astype(float), (df_cust_item_qty['cust_id'],df_cust_item_qty['item_id'])))

In [32]:
sparse_customer_item

<4017x3671 sparse matrix of type '<class 'numpy.float64'>'
	with 266596 stored elements in Compressed Sparse Row format>

In [33]:
model = implicit.als.AlternatingLeastSquares(num_threads=1)

In [34]:
model.fit(sparse_customer_item)

  0%|          | 0/15 [00:00<?, ?it/s]

## Generate Similar Items

In [35]:
ref_item_id = df_filter_cust_item['item_id'].unique()

In [36]:
item_arr, score_arr = model.similar_items(ref_item_id, N=10)

In [37]:
df_item_temp = pd.DataFrame(item_arr)

In [38]:
df_item_temp['Ref Item ID'] = ref_item_id

In [39]:
df_item_temp.head()

,0,1,2,3,4,5,6,7,8,9,Ref Item ID
0,31,2830,2505,1561,2854,3103,1879,2915,2479,3511,31
1,2761,2265,2296,1335,220,74,1584,155,1522,3151,2761
2,1334,316,2814,344,2105,1890,1085,2033,2876,2068,1334
3,1333,2733,2736,1601,1896,3461,2794,772,2569,2281,1333
4,1790,1125,856,944,347,1336,1673,1726,749,2462,1790


In [40]:
df_item_rank= pd.melt(df_item_temp,
                  id_vars = ['Ref Item ID'],
                  var_name = ['Item Rank'],
                  value_name = 'Related Item ID')

In [41]:
df_item_rank.sort_values(['Ref Item ID','Item Rank']).head(20)

,Ref Item ID,Item Rank,Related Item ID
3239,0,0,0
6910,0,1,57
10581,0,2,260
14252,0,3,2807
17923,0,4,130
21594,0,5,293
25265,0,6,1863
28936,0,7,122
32607,0,8,1709
36278,0,9,127


In [42]:
 df_score_temp = pd.DataFrame(score_arr)

In [43]:
df_score_temp['Ref Item ID'] = ref_item_id

In [44]:
df_score_rank= pd.melt(df_score_temp,
                  id_vars = ['Ref Item ID'],
                  var_name = ['Item Rank'],
                  value_name = 'Score')

In [45]:
df_score_rank.sort_values(['Ref Item ID','Item Rank']).head(20)

,Ref Item ID,Item Rank,Score
3239,0,0,1.000000
6910,0,1,0.729857
10581,0,2,0.681337
14252,0,3,0.533633
17923,0,4,0.451290
21594,0,5,0.447868
25265,0,6,0.442458
28936,0,7,0.433937
32607,0,8,0.431659
36278,0,9,0.430151


In [46]:
df_item_score = df_item_rank.merge(df_score_rank,
                                   how='inner',
                                   on=['Ref Item ID','Item Rank'])

In [47]:
df_item_score.sort_values(['Ref Item ID','Item Rank'],
                          inplace=True)

In [48]:
df_item_score.head(20)

,Ref Item ID,Item Rank,Related Item ID,Score
3239,0,0,0,1.000000
6910,0,1,57,0.729857
10581,0,2,260,0.681337
14252,0,3,2807,0.533633
17923,0,4,130,0.451290
21594,0,5,293,0.447868
25265,0,6,1863,0.442458
28936,0,7,122,0.433937
32607,0,8,1709,0.431659
36278,0,9,127,0.430151


In [55]:
df_item_desc = df_filter_cust_item[['item_id','StockCode', 'Description']].copy()
df_item_desc.drop_duplicates(inplace=True)

In [56]:
df_similar_item_temp = df_item_score.merge(df_item_desc,
                    how="left",
                    left_on="Ref Item ID",
                    right_on="item_id")

In [57]:
df_similar_item_temp.drop(columns=['item_id'], inplace=True)

In [58]:
df_similar_item_temp.columns = ['Ref Item ID', 'Item Rank', 
                                'Related Item ID', 'Score', 
                                'Ref_StockCode','Ref_Description']

In [59]:
df_similar_item_temp.shape

(39020, 6)

In [60]:
df_similar_item_temp.head()

,Ref Item ID,Item Rank,Related Item ID,Score,Ref_StockCode,Ref_Description
0,0,0,0,1.000000,85123A,WHITE HANGING HEART T-LIGHT HOLDER
1,0,0,0,1.000000,85123A,CREAM HANGING HEART T-LIGHT HOLDER
2,0,1,57,0.729857,85123A,WHITE HANGING HEART T-LIGHT HOLDER
3,0,1,57,0.729857,85123A,CREAM HANGING HEART T-LIGHT HOLDER
4,0,2,260,0.681337,85123A,WHITE HANGING HEART T-LIGHT HOLDER


In [63]:
df_similar_item = df_similar_item_temp.merge(df_item_desc,
                    how="left",
                    left_on="Related Item ID",
                    right_on="item_id")

In [64]:
df_similar_item.drop(columns=['item_id'], inplace=True)


In [65]:
df_similar_item_temp.columns

Index(['Ref Item ID', 'Item Rank', 'Related Item ID', 'Score', 'Ref_StockCode',
       'Ref_Description'],
      dtype='object')

In [66]:
df_similar_item_temp.shape

(39020, 6)